In [11]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
min_loss = 0
max_profit = 0
buy_times = 0


In [12]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [13]:
symbols = """
select distinct(name) from stockdatadaily order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [14]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [15]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = 0.0
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    training = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        self.training = list()

sql1= """
select distinct(name) from stockdatadaily order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)



<h1> Updating resources

In [16]:

symbol_list = random.sample(symbol_list, 100)
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdatadaily where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done


In [17]:
sql_dates = """
select distinct(date) from stockdatadaily order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Trading strategy with the training sets generation

In [18]:

def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global min_loss
    global max_profit
    global buy_times
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatadaily where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):

                        
                        stop_order = current_symbol.support * 0.97  
                        quantity = (0.02 * remaining_cash) // (current_price - stop_order)
                        if quantity > 1:
                            current_symbol.bought = current_price
                            remaining_cash = remaining_cash - commission - (quantity * current_price)                        
                            position[symbol] = quantity
                            current_symbol.times_bought += 1
                            print("Buying ", quantity, "stocks of ", symbol, "At price", current_symbol.bought, "on date", date)                                      
                            buy_times += 1
                            current_symbol.stop_order = stop_order
                        
                        
                        
                        training_inst = training_set_extractor(data_till_date, date)                            
                        current_symbol.training.append(training_inst)

                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                current_symbol.training[-1][-1] = 1
                                win += 1                                
                                current_symbol.flag_for_wins = True
                                
                    if current_symbol.stop_order > current_price:
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            current_symbol.training[-1][-1]= 1
                            if current_symbol.flag_for_wins == False:
                                win += 1
                            current_symbol.flag_for_wins = False
                         
                        else:
                            loss += 1
                            current_symbol.training[-1][-1] = 0
                            current_symbol.flag_for_wins = False
                        
                        profit += profit_today
            
                        if profit_today < min_loss:
                            min_loss = profit_today
                        if profit_today > max_profit:
                            max_profit = profit_today
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [19]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def training_set_extractor(data, date):
    lastFrom = pd.to_datetime(date)

    destDate = lastFrom - pd.Timedelta(days=30)
    lastFrom = str(lastFrom.date())
    destDate = str(destDate.date())

    df = pd.DataFrame()
    df = data[(data['date'] >= destDate) & (data['date'] <= lastFrom)]

    df = df.set_index('date')

    df.index = pd.DatetimeIndex(df.index)
    idx = pd.date_range(destDate, lastFrom)
    df = df.reindex(idx, fill_value=0)
    
    highs = df['2. high'].tolist()
    volumes = df['5. volume'].tolist()
    lows = df['3. low'].tolist()
    
    for i in range(1, len(highs)):
        if highs[i] == 0:
            highs[i] = highs[i-1]
        if volumes[i] == 0:
            volumes[i] = volumes[i-1]
        if lows[i] == 0:
            lows[i] = lows[i-1]
        
    
    training_instance = [highs, volumes, lows, 'na']
    
    return training_instance[:] 
   

In [20]:
automatic_trader(symbol_list)

Buying  69.0 stocks of  CP At price 22.94 on date 2000-01-06
Buying  102.0 stocks of  MCY At price 22.62 on date 2000-01-06
Buying  12.0 stocks of  PG At price 109.4 on date 2000-01-06
Buying  37.0 stocks of  CATY At price 40.13 on date 2000-01-07
Buying  6.0 stocks of  UTX At price 64.69 on date 2000-01-07
Buying  6.0 stocks of  LSCC At price 48.0 on date 2000-01-10
Buying  53.0 stocks of  AVD At price 6.25 on date 2000-01-10
Buying  57.0 stocks of  EBF At price 8.25 on date 2000-01-10
Buying  10.0 stocks of  SHI At price 18.5 on date 2000-01-10
Buying  25.0 stocks of  PFH At price 18.25 on date 2000-01-10
Buying  8.0 stocks of  IDN At price 12.25 on date 2000-01-10
Buying  2.0 stocks of  VOXX At price 34.25 on date 2000-01-11
Buying  32.0 stocks of  IRET At price 7.969 on date 2000-01-11
Buying  2.0 stocks of  TESS At price 21.0 on date 2000-01-12
Buying  72.0 stocks of  VTNR At price 0.2204 on date 2000-01-13
Buying  7.0 stocks of  LDF At price 11.25 on date 2000-01-13
Buying  17.0 

Buying  2.0 stocks of  CP At price 106.96 on date 2013-01-02
Buying  4.0 stocks of  CHUY At price 33.52 on date 2013-06-06
Buying  2.0 stocks of  PRO At price 32.315 on date 2013-07-22
Selling  2.0 stocks of SYNC  at price 2.94 on date 2013-08-12
Selling  20.0 stocks of BNO  at price 42.87 on date 2013-09-27
Buying  4.0 stocks of  FDS At price 113.298 on date 2013-10-04
Buying  27.0 stocks of  RIGS At price 24.73 on date 2013-10-14
Buying  11.0 stocks of  GMZ At price 20.6 on date 2013-12-05
Buying  4.0 stocks of  FITBI At price 25.68 on date 2014-01-10
Buying  2.0 stocks of  JMPB At price 26.27 on date 2014-01-24
Selling  4.0 stocks of FDS  at price 104.055 on date 2014-02-14
Buying  3.0 stocks of  SCIF At price 31.98 on date 2014-03-04
Buying  17.0 stocks of  QINC At price 20.06 on date 2014-03-17
Selling  9.0 stocks of LBTYK  at price 43.83 on date 2014-05-27
Selling  2.0 stocks of PRO  at price 22.1 on date 2014-06-03
Buying  2.0 stocks of  FEEU At price 133.469 on date 2014-06-05


In [21]:
print("Remaining Cash ::" , remaining_cash)

Remaining Cash :: 951.7369000000024


In [22]:
current_profit = 0
total_valuation = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    total_valuation += position[key] * stock_equities[key].current_price
    
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Total valuation", total_valuation)

Profit -2119.7409000000002
Remaining cash  951.7369000000024
Total valuation 10569.098599999996


<h1> Extracting training sets from the trading strategy's cache

In [23]:
training = []
for i in symbol_list:
    if(stock_equities[i].training):
        if stock_equities[i].training[-1][-1] == 'na':
            del stock_equities[i].training[-1]
        training.extend(stock_equities[i].training)
        
X = []
Y = []


random.shuffle(training)

for i in training:
    X.append([i[0], i[1]])
    Y.append([i[3]])

<h1> Saving training sets 

In [17]:
import pickle
pickle_out = open("Y30.pickle", "wb")
pickle.dump(Y, pickle_out)
pickle_out.close()


pickle_out = open("X30.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

In [18]:
len(Y)

6475

In [25]:
training[1]

[[156.85,
  155.49,
  152.78,
  151.4,
  151.4,
  151.4,
  138.0,
  139.0,
  140.39,
  139.57,
  145.29,
  145.29,
  145.29,
  145.29,
  153.0,
  156.69,
  157.6499,
  155.27,
  155.27,
  155.27,
  162.54,
  160.6,
  160.426,
  146.89,
  142.6,
  142.6,
  142.6,
  136.67,
  140.14,
  141.6852,
  141.215],
 [914242.0,
  1209776.0,
  715277.0,
  1172818.0,
  1172818.0,
  1172818.0,
  1260752.0,
  1152861.0,
  621904.0,
  616622.0,
  1229137.0,
  1229137.0,
  1229137.0,
  1229137.0,
  1797837.0,
  1121975.0,
  758459.0,
  799559.0,
  799559.0,
  799559.0,
  945771.0,
  802471.0,
  848246.0,
  3677967.0,
  1642103.0,
  1642103.0,
  1642103.0,
  1318964.0,
  1341603.0,
  1476499.0,
  1025636.0],
 [154.12,
  146.16,
  148.6,
  138.88,
  138.88,
  138.88,
  130.1,
  129.6,
  137.505,
  132.5,
  137.29,
  137.29,
  137.29,
  137.29,
  143.42,
  148.035,
  150.49,
  150.5935,
  150.5935,
  150.5935,
  155.0,
  154.8,
  152.0,
  132.0,
  135.32,
  135.32,
  135.32,
  134.14,
  135.01,
  132.6,
 